In [1]:
import sys
sys.path.append("/home/jchishol/TRecNet")
sys.path.append("home/jchishol/")

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv1D, Flatten, Dense, Input, concatenate, Masking, LSTM, TimeDistributed, Lambda, Reshape, Multiply, BatchNormalization, Bidirectional
from keras import regularizers 
from keras import initializers
from sklearn.model_selection import train_test_split
import h5py
import os 
from clr_callback import *
from keras.callbacks import TensorBoard
import keras.backend as K  
from keras.optimizers import * 
# import keras_one_cycle_clr as ktool
import importlib
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import normalize
import shape_timesteps
from argparse import ArgumentParser

In [2]:

model_name = 'TRecNet'

# Dataset to train and test on
dataset = h5py.File('/mnt/xrootdg/jchishol/mntuples_08_01_22/variables_ttbar_ljets_6j_train.h5','r')

# Numpy dictionary of [max,mean] for each variable
X_maxmean_dic = np.load('/home/jchishol/TRecNet/X_maxmean_variables_ttbar_ljets_6j_train.npy',allow_pickle=True).item()
Y_maxmean_dic = np.load('/home/jchishol/TRecNet/Y_maxmean_variables_ttbar_ljets_6j_train.npy',allow_pickle=True).item()

# Get the maxmean values we actually need for this particular model
xmm_keys = list(X_maxmean_dic.keys())
if model_name == 'TRecNet':
    ymm_keys = [key for key in Y_maxmean_dic.keys() if 'isTruth' not in key and 'ttbar' not in key]
elif model_name == 'TRecNet+ttbar' or model_name == 'TRecNet+ttbar+JetPretrain':
    ymm_keys = [key for key in Y_maxmean_dic.keys() if 'isTruth' not in key]
elif model_name == 'JetPretrainer':
    ymm_keys = [key for key in Y_maxmean_dic.keys() if 'isTruth' in key]
X_maxmean = np.array([X_maxmean_dic[key] for key in xmm_keys])
Y_maxmean = np.array([Y_maxmean_dic[key] for key in ymm_keys])

# X and Y variables to be used (important that these are in the correct order for the helper code)
X_keys = ['j1_pt', 'j1_eta', 'j1_phi', 'j1_m', 'j1_isbtag', 'j2_pt', 'j2_eta', 'j2_phi', 'j2_m', 'j2_isbtag', 'j3_pt', 'j3_eta', 'j3_phi', 'j3_m', 'j3_isbtag', 'j4_pt', 'j4_eta', 'j4_phi', 'j4_m', 'j4_isbtag', 'j5_pt', 'j5_eta', 'j5_phi', 'j5_m', 'j5_isbtag', 'j6_pt', 'j6_eta', 'j6_phi', 'j6_m', 'j6_isbtag', 'lep_pt', 'lep_eta', 'lep_phi', 'met_met', 'met_phi']
Y_keys = ['th_pt', 'th_eta','th_phi','th_m', 'wh_pt', 'wh_eta', 'wh_phi', 'wh_m', 'tl_pt', 'tl_eta', 'tl_phi', 'tl_m', 'wl_pt', 'wl_eta', 'wl_phi', 'wl_m']
if 'ttbar' in model_name:
    Y_keys.extend(['ttbar_pt','ttbar_eta','ttbar_phi','ttbar_m'])
if model_name=='JetPretrainer': 
    Y_keys = ['j1_isTruth','j2_isTruth','j3_isTruth','j4_isTruth','j5_isTruth','j6_isTruth']

In [39]:
import normalize
phi_keys = list(filter(lambda a: 'phi' in a, dataset.keys()))
crop0 = np.array(dataset.get('th_pt')).size
Old_Scaler = normalize.Scale_variables(phi_keys,dataset,crop0)
old_data, old_names = Old_Scaler.scale_arrays(dataset,Y_keys,Y_maxmean,crop0)

In [41]:
print(old_data)
print(np.array(Y_df))

[[ 0.05732365  0.11583149 -0.0500405  ...  0.03451359 -0.19803211
   0.00935271]
 [-0.01164451  0.05001605 -0.00413855 ...  0.0033732   0.2936836
   0.08307171]
 [-0.00944873  0.05146901 -0.01364326 ...  0.01639921 -0.12610883
   0.00259989]
 ...
 [-0.0519874  -0.00840842  0.00045205 ... -0.02929574  0.02099547
   0.01005571]
 [ 0.04423211  0.00306726  0.15715494 ... -0.03441462  0.09351998
  -0.00040052]
 [-0.0353758   0.01258297 -0.03226203 ... -0.03332592  0.00307783
  -0.00429655]]
[[ 0.05732365  0.11583149 -0.0500405  ...  0.03451359 -0.19803211
   0.00935271]
 [-0.01164451  0.05001605 -0.00413855 ...  0.0033732   0.2936836
   0.08307171]
 [-0.00944873  0.05146901 -0.01364326 ...  0.01639921 -0.12610883
   0.00259989]
 ...
 [-0.0519874  -0.00840842  0.00045205 ... -0.02929574  0.02099547
   0.01005571]
 [ 0.04423211  0.00306726  0.15715494 ... -0.03441462  0.09351998
  -0.00040052]
 [-0.0353758   0.01258297 -0.03226203 ... -0.03332592  0.00307783
  -0.00429655]]


In [3]:
import normalize_new
importlib.reload(normalize_new)
New_Scaler = normalize_new.Scaler()
#X_df = New_Scaler.scale_arrays(dataset, X_keys, X_maxmean_dic)
Y_df = New_Scaler.scale_arrays(dataset, Y_keys, Y_maxmean_dic)

Creating Scaler.


In [5]:
Y_df

,th_pt,th_px,th_py,th_eta,th_m,wh_pt,wh_px,wh_py,wh_eta,wh_m,tl_pt,tl_px,tl_py,tl_eta,tl_m,wl_pt,wl_px,wl_py,wl_eta,wl_m
0,0.057324,0.115831,-0.050040,-0.038596,0.004687,0.021417,0.065337,-0.071022,-0.064886,-0.022141,0.029848,0.142807,0.004203,-0.159268,-0.001852,0.031179,0.101608,0.034514,-0.198032,0.009353
1,-0.011645,0.050016,-0.004139,0.030934,-0.010892,-0.040038,0.004225,-0.028878,0.079537,0.004648,-0.024728,-0.071331,0.020027,0.074775,-0.001803,-0.064117,-0.000316,0.003373,0.293684,0.083072
2,-0.009449,0.051469,-0.013643,0.021860,0.001218,-0.013166,0.043444,0.037189,0.062836,-0.003239,-0.052461,-0.039742,-0.009570,-0.165809,-0.001443,-0.046625,0.014643,0.016399,-0.126109,0.002600
3,-0.041773,0.012855,-0.020282,0.266364,-0.004010,-0.005188,0.054380,-0.035007,0.188943,0.015466,0.018438,-0.112599,-0.051888,0.002125,-0.000879,0.016400,-0.091592,-0.012632,-0.036282,0.001031
4,-0.026526,0.020599,0.041398,0.000216,0.016248,0.003348,0.013656,0.080168,0.038674,-0.009136,-0.038487,0.016252,-0.045645,-0.027249,-0.032493,-0.021060,-0.031781,-0.038902,0.003920,0.007350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20490861,0.074511,0.025839,-0.198188,0.085964,0.001487,0.067981,0.047758,-0.148772,0.129821,0.003020,0.061885,-0.035261,0.144412,0.005141,0.004661,0.058619,-0.031664,0.127691,0.040472,0.007850
20490862,0.098496,-0.130201,-0.142462,-0.121917,0.004719,0.152379,-0.195612,-0.132486,-0.126397,0.009605,0.179973,0.237284,0.183936,0.017442,-0.015852,0.117321,0.110208,0.161027,0.023790,-0.005174
20490863,-0.051987,-0.008408,0.000452,0.295700,0.000752,-0.025641,0.019474,-0.040926,0.168955,0.007684,-0.068415,0.010595,-0.015140,-0.083631,-0.001408,-0.028473,-0.030954,-0.029296,0.020995,0.010056
20490864,0.044232,0.003067,0.157155,0.020678,0.012889,0.047808,-0.042334,0.125787,0.009619,0.000068,-0.017675,0.036128,-0.061747,0.108571,-0.001179,0.003079,0.068636,-0.034415,0.093520,-0.000401


In [9]:
importlib.reload(normalize_new)
New_Scaler = normalize_new.Scaler()
#x_orig = New_Scaler.invscale_arrays(np.array(X_df),X_df.keys(),X_maxmean_dic)
y_orig = New_Scaler.invscale_arrays(np.array(Y_df),Y_df.keys(),Y_maxmean_dic)

Creating Scaler.


In [38]:
np.allclose(np.array(y_orig['th_pt']),np.array(dataset.get('th_pt')),rtol=0,atol=1e-12)
# Good enough!

True

In [16]:
len(data_arr)==len(old_data)

True

In [20]:
print(names)
print(old_names)

['j1_pt', 'j1_px', 'j1_py', 'j1_eta', 'j1_m', 'j1_isbtag', 'j2_pt', 'j2_px', 'j2_py', 'j2_eta', 'j2_m', 'j2_isbtag', 'j3_pt', 'j3_px', 'j3_py', 'j3_eta', 'j3_m', 'j3_isbtag', 'j4_pt', 'j4_px', 'j4_py', 'j4_eta', 'j4_m', 'j4_isbtag', 'j5_pt', 'j5_px', 'j5_py', 'j5_eta', 'j5_m', 'j5_isbtag', 'j6_pt', 'j6_px', 'j6_py', 'j6_eta', 'j6_m', 'j6_isbtag', 'lep_pt', 'lep_px', 'lep_py', 'lep_eta', 'met_met', 'met_phi-sin', 'met_phi-cos']
['j1_ptbox', 'j1_px', 'j1_py', 'j1_eta', 'j1_m', 'j1_isbtag', 'j2_ptbox', 'j2_px', 'j2_py', 'j2_eta', 'j2_m', 'j2_isbtag', 'j3_ptbox', 'j3_px', 'j3_py', 'j3_eta', 'j3_m', 'j3_isbtag', 'j4_ptbox', 'j4_px', 'j4_py', 'j4_eta', 'j4_m', 'j4_isbtag', 'j5_ptbox', 'j5_px', 'j5_py', 'j5_eta', 'j5_m', 'j5_isbtag', 'j6_ptbox', 'j6_px', 'j6_py', 'j6_eta', 'j6_m', 'j6_isbtag', 'lep_ptbox', 'lep_px', 'lep_py', 'lep_eta', 'met_met', 'met_phi-sin', 'met_phi-cos']


In [21]:
for i in range(len(data_arr)):   # Each event
    for j in range(len(data_arr[i])):   # Each variable
        if data_arr[i][j]!=old_data[i][j]:
            print(names[j])
            print('Old:',old_data[i][j])
            print('New:',data_arr[i][j])

met_phi-sin
Old: 0.6530058515208241
New: 0.6530058515323867
met_phi-cos
Old: 0.14456373392582628
New: 0.1445637339224435
met_phi-cos
Old: -0.0008557746395469734
New: -0.0008557746395469727
met_phi-cos
Old: -0.38990857846925997
New: -0.38990858624183683
met_phi-sin
Old: -0.5789911339350995
New: -0.57899113395539
met_phi-sin
Old: 0.2471108559129919
New: 0.2471108559107161
met_phi-cos
Old: 0.8989328370773793
New: 0.8989328370776748
met_phi-sin
Old: 0.5772331950445262
New: 0.5772331950450105
met_phi-sin
Old: 0.43684152024832695
New: 0.4368415202771274
met_phi-sin
Old: -0.5478400071638715
New: -0.5478400071887959
met_phi-sin
Old: -0.467592871616718
New: -0.46759287354022383
met_phi-sin
Old: -0.14278170974371315
New: -0.1427817097404519
met_phi-sin
Old: 0.6985799852053334
New: 0.6985799851976848
met_phi-sin
Old: -0.16388760140860995
New: -0.1638876014037175
met_phi-sin
Old: 0.753390755706563
New: 0.753390755710791
met_phi-sin
Old: 0.7602687290803812
New: 0.7602689838779659
met_phi-sin
Old: 0

KeyboardInterrupt: 

In [37]:
for i in range(len(names)):
    print(names[i])
    bad = np.where(data_arr[:,i]!=old_data[:,i])[0]
    print(bad)
    if len(bad)>0:
        print(len(bad))
        for item in bad:
            print(data_arr[item][i])
            print(old_data[item][i])
# I'm satisfied that these are close enough

j1_pt
[]
j1_px
[]
j1_py
[]
j1_eta
[]
j1_m
[]
j1_isbtag
[]
j2_pt
[]
j2_px
[]
j2_py
[12343909]
1
-0.028223700350202146
-0.028223700350237396
j2_eta
[]
j2_m
[]
j2_isbtag
[]
j3_pt
[]
j3_px
[]
j3_py
[]
j3_eta
[]
j3_m
[]
j3_isbtag
[]
j4_pt
[]
j4_px
[]
j4_py
[]
j4_eta
[]
j4_m
[]
j4_isbtag
[]
j5_pt
[]
j5_px
[]
j5_py
[]
j5_eta
[]
j5_m
[]
j5_isbtag
[]
j6_pt
[]
j6_px
[]
j6_py
[]
j6_eta
[]
j6_m
[]
j6_isbtag
[]
lep_pt
[]
lep_px
[16620477]
1
0.033757773349372786
0.033616372764351914
lep_py
[]
lep_eta
[]
met_met
[]
met_phi-sin
[ 1179583  3063615  3235775  4943359  4998559  5168063  5676991  6447903
  7028767  7997279  8001471  8119391  9265087  9422375 11221951 12081719
 12379191 14973247 17931359]
19
0.6530058515323867
0.6530058515208241
-0.57899113395539
-0.5789911339350995
0.2471108559107161
0.2471108559129919
0.5772331950450105
0.5772331950445262
0.4368415202771274
0.43684152024832695
-0.5478400071887959
-0.5478400071638715
-0.46759287354022383
-0.467592871616718
-0.1427817097404519
-0.1427817097

In [15]:
import shape_timesteps
timestep_builder = shape_timesteps.Shape_timesteps()
old_totalX_jets, old_totalX_other = timestep_builder.reshape_X(dataset, old_data, old_names, phi_keys,crop0,False,True)


In [19]:
import shape_timesteps_new
importlib.reload(shape_timesteps_new)
new_timestep_builder = shape_timesteps_new.Shape_timesteps()
totalX_jets, totalX_other = new_timestep_builder.reshape_X(dataset, data)

Jet Names: ['j1_pt', 'j1_px', 'j1_py', 'j1_eta', 'j1_m', 'j1_isbtag', 'j2_pt', 'j2_px', 'j2_py', 'j2_eta', 'j2_m', 'j2_isbtag', 'j3_pt', 'j3_px', 'j3_py', 'j3_eta', 'j3_m', 'j3_isbtag', 'j4_pt', 'j4_px', 'j4_py', 'j4_eta', 'j4_m', 'j4_isbtag', 'j5_pt', 'j5_px', 'j5_py', 'j5_eta', 'j5_m', 'j5_isbtag', 'j6_pt', 'j6_px', 'j6_py', 'j6_eta', 'j6_m', 'j6_isbtag']
Other Names: ['lep_pt', 'lep_px', 'lep_py', 'lep_eta', 'met_met', 'met_phi-sin', 'met_phi-cos']
Num Jet Features: 6
Num Jets: 6.0
Jet pt names: ['j1_pt', 'j2_pt', 'j3_pt', 'j4_pt', 'j5_pt', 'j6_pt']
Jet Existence Dic: {'j1_exists': array([1, 1, 1, ..., 1, 1, 1]), 'j2_exists': array([1, 1, 1, ..., 1, 1, 1]), 'j3_exists': array([1, 1, 1, ..., 1, 1, 1]), 'j4_exists': array([1, 1, 1, ..., 1, 1, 1]), 'j5_exists': array([1, 0, 1, ..., 0, 0, 0]), 'j6_exists': array([0, 0, 1, ..., 0, 0, 0])}


In [30]:
old_totalX_jets.shape

(20490866, 6, 6)

In [31]:
totalX_jets.shape

(20490866, 6, 6)

In [34]:
old_totalX_other.shape==totalX_other.shape


True

In [40]:
importlib.reload(normalize_new)
Scaler = normalize_new.Scaler()
Y_df = Scaler.scale_arrays(dataset, Y_keys, Y_maxmean_dic)

Creating Scaler.


In [42]:
old_Y, old_Ynames = old_Scaler.scale_arrays(dataset,Y_keys,Y_maxmean,crop0)

In [61]:
print(np.array(Y_df))
print(old_Y)

[[ 0.05732365  0.11583149 -0.0500405  ...  0.03451359 -0.19803211
   0.00935271]
 [-0.01164451  0.05001605 -0.00413855 ...  0.0033732   0.2936836
   0.08307171]
 [-0.00944873  0.05146901 -0.01364326 ...  0.01639921 -0.12610883
   0.00259989]
 ...
 [-0.0519874  -0.00840842  0.00045205 ... -0.02929574  0.02099547
   0.01005571]
 [ 0.04423211  0.00306726  0.15715494 ... -0.03441462  0.09351998
  -0.00040052]
 [-0.0353758   0.01258297 -0.03226203 ... -0.03332592  0.00307783
  -0.00429655]]
[[ 0.05732365  0.11583149 -0.0500405  ...  0.03451359 -0.19803211
   0.00935271]
 [-0.01164451  0.05001605 -0.00413855 ...  0.0033732   0.2936836
   0.08307171]
 [-0.00944873  0.05146901 -0.01364326 ...  0.01639921 -0.12610883
   0.00259989]
 ...
 [-0.0519874  -0.00840842  0.00045205 ... -0.02929574  0.02099547
   0.01005571]
 [ 0.04423211  0.00306726  0.15715494 ... -0.03441462  0.09351998
  -0.00040052]
 [-0.0353758   0.01258297 -0.03226203 ... -0.03332592  0.00307783
  -0.00429655]]


In [69]:
Y_data = np.array(Y_df)
for i in range(len(old_Ynames)):
    #print(old_Ynames[i])
    bad = np.where(Y_data[:,i]!=old_Y[:,i])[0]
    #print(bad)
    if len(bad)>0:
        #print(len(bad))
        print(old_Ynames[i])
        print(bad)
        for item in bad:
            print(Y_data[item][i])
            print(old_Y[item][i])

# Close enough I guess

tl_py
[2528451 2998095]
0.032716745109425925
0.03271674558496309
-0.03888861142346341
-0.039540271117152054
wl_py
[14949519]
-0.006250717264092814
-0.006929255566194777
